
# Ceph Manager — Operator Notebook

Use this notebook to **add/delete CephX users** and **add/resize/delete CephFS subvolumes**, either **one at a time** or **in bulk from CSV files**.

This notebook expects the Python client `CephManagerClient` to be importable (from your package or as a module).


In [ ]:
from typing import Any, Dict, List, Optional
from dataclasses import dataclass
import json
import pandas as pd
from fabric_ceph_client.fabric_ceph_client import CephManagerClient, ApiError

In [ ]:

BASE_URL = "https://23.134.232.211"   # e.g., "https://mgr.example.org/api"
TOKEN = "/Users/kthare10/work/id_token_prod.json"                           # paste a JWT if your service requires it, else None
VERIFY_TLS = False                      # set False only for testing self-signed certs

client = CephManagerClient(base_url=BASE_URL, token_file=TOKEN, verify=VERIFY_TLS)


## CephX user helpers
Single operations and CSV batch upload.


In [ ]:
def list_cluster_info() -> Dict[str, Any]:
    try:
        return client.list_cluster_info()
    except ApiError as e:
        print("API error during list_cluster_info:", e)
        raise

def delete_user(entity: str, x_cluster: Optional[str]=None) -> Dict[str, Any]:
    try:
        return client.delete_user(entity, x_cluster=x_cluster)
    except ApiError as e:
        print("API error during delete_user:", e)
        raise

def export_keyrings(entities: List[str], x_cluster: Optional[str]=None) -> str:
    try:
        return client.export_users(entities, x_cluster=x_cluster)
    except ApiError as e:
        print("API error during export_keyrings:", e)
        raise

def list_users() -> str:
    try:
        return client.list_users()
    except ApiError as e:
        print("API error during list_users:", e)
        raise


## CephFS subvolume helpers
Create/resize/delete subvolumes and check/get info. `mode` is used on **create**; `size` sets quota in bytes.


In [ ]:

def ensure_group(vol_name: str, group_name: str, x_cluster: Optional[str]=None) -> Dict[str, Any]:
    try:
        return client.create_subvolume_group(vol_name, group_name, x_cluster=x_cluster)
    except ApiError as e:
        print("API error during ensure_group:", e)
        raise

def create_or_resize_subvol(vol_name: str, subvol_name: str, *, group_name: Optional[str]=None,
                            size_bytes: Optional[int]=None, mode: Optional[str]=None,
                            x_cluster: Optional[str]=None) -> Dict[str, Any]:
    try:
        return client.create_or_resize_subvolume(vol_name, subvol_name, group_name=group_name,
                                                 size=size_bytes, mode=mode, x_cluster=x_cluster)
    except ApiError as e:
        print("API error during create_or_resize_subvol:", e)
        raise

def get_subvol_info(vol_name: str, subvol_name: str, *, group_name: Optional[str]=None,
                    x_cluster: Optional[str]=None) -> Dict[str, Any]:
    try:
        return client.get_subvolume_info(vol_name, subvol_name, group_name=group_name, x_cluster=x_cluster)
    except ApiError as e:
        print("API error during get_subvol_info:", e)
        raise

def subvol_exists(vol_name: str, subvol_name: str, *, group_name: Optional[str]=None,
                  x_cluster: Optional[str]=None) -> bool:
    try:
        return client.subvolume_exists(vol_name, subvol_name, group_name=group_name, x_cluster=x_cluster)
    except ApiError as e:
        print("API error during subvol_exists:", e)
        raise

def delete_subvol(vol_name: str, subvol_name: str, *, group_name: Optional[str]=None, force: bool=False,
                  x_cluster: Optional[str]=None) -> Dict[str, Any]:
    try:
        return client.delete_subvolume(vol_name, subvol_name, group_name=group_name, force=force, x_cluster=x_cluster)
    except ApiError as e:
        print("API error during delete_subvol:", e)
        raise



## One-at-a-time examples
Uncomment and run what you need.


In [56]:
print(list_users())

/Users/kthare10/renci/code/fabric/1.6/fabric_ceph/.venv/lib/python3.13/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host '23.134.232.211'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/Users/kthare10/renci/code/fabric/1.6/fabric_ceph/.venv/lib/python3.13/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host '23.134.232.211'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/Users/kthare10/renci/code/fabric/1.6/fabric_ceph/.venv/lib/python3.13/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host '23.134.232.211'. Adding certificate verification is strongly advised. See: https://urllib3.readthedoc

API error during list_users: [500] https://23.134.232.211/cluster/user :: Internal Server Error


ApiError: [500] https://23.134.232.211/cluster/user :: Internal Server Error

In [ ]:
result = create_or_resize_subvol("CEPH-FS-01", "kthare10_0011904101", size_bytes=10 * 1024**3,  mode="0777")

In [ ]:
result = create_or_resize_subvol("CEPH-FS-01", "pruth_0031379841", size_bytes=10 * 1024**3,  mode="0777")

In [ ]:
print(result)
paths = result.get("data")[0].get("details").get("paths")
print(paths)


In [ ]:
tmpl_caps = [
    {"entity": "mon", "cap": "allow r fsname={fs}"},
    {"entity": "mds", "cap": "allow rw fsname={fs} path={path}"},
    {"entity": "osd", "cap": "allow rw tag cephfs data={fs}"},
    {"entity": "osd", "cap": "allow rw tag cephfs metadata={fs}"},
]
try:
    res = client.apply_user_templated(
        user_entity="client.pruth_0031379841",
        template_capabilities=tmpl_caps,
        fs_name="CEPH-FS-01",
        subvol_name="pruth_0031379841",
        #group_name="fabric_staff",      # optional
        sync_across_clusters=True,      # keep SAME secret everywhere
        #preferred_source="europe",      # where to create/update first (if user doesn't exist)
        # x_cluster="europe,lab"        # optional per-call override
    )
    # 4) Inspect result
    print("Source cluster:", res.get("source_cluster"))
    print("Imported to:", res.get("imported_to", []))
    print("Per-cluster paths:", res.get("paths", {}))
    print("Per-cluster caps applied:", res.get("caps_applied", {}))
    print("Errors (if any):", res.get("errors", {}))

except ApiError as e:
    print("API error:", e.status, e.url, e.message)
    print("Payload:", e.payload)

In [50]:
keyring = export_keyrings(["client.kthare10_0011904101", "client.pruth_0031379841"])
print(keyring)

/Users/kthare10/renci/code/fabric/1.6/fabric_ceph/.venv/lib/python3.13/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host '23.134.232.211'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


{'clusters': {'east': {'client.kthare10_0011904101': '"[client.kthare10_0011904101]\\n\\tkey = AQApM+BoJQCNABAA+id/X+Jnftn2Gud7v3VLvw==\\n\\tcaps mds = \\"allow rw fsname=CEPH-FS-01 path=/volumes/_nogroup/kthare10_0011904101/454d15fd-e116-4af1-b743-f4bba6bfb8e5\\"\\n\\tcaps mon = \\"allow r fsname=CEPH-FS-01\\"\\n\\tcaps osd = \\"allow rw tag cephfs data=CEPH-FS-01\\"\\n\\n"', 'client.pruth_0031379841': '"[client.pruth_0031379841]\\n\\tkey = AQAXPuBoDvsMLRAA/wY0kk2WrPXfrlysgUohAg==\\n\\tcaps mds = \\"allow rw fsname=CEPH-FS-01 path=/volumes/_nogroup/pruth_0031379841/94bbbfea-db02-4539-a252-d3dd6bbaa713\\"\\n\\tcaps mon = \\"allow r fsname=CEPH-FS-01\\"\\n\\tcaps osd = \\"allow rw tag cephfs data=CEPH-FS-01\\"\\n\\n"'}}, 'size': 1, 'status': 200, 'type': 'keyring'}


In [54]:
info = get_subvol_info("CEPH-FS-01", "kthare10_0011904101"); 
info

/Users/kthare10/renci/code/fabric/1.6/fabric_ceph/.venv/lib/python3.13/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host '23.134.232.211'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


{'data': [{'details': {'east': {'atime': '2025-10-03 16:51:06',
     'bytes_pcent': '0.00',
     'bytes_quota': 10737418240,
     'bytes_used': 0,
     'created_at': '2025-10-03 16:51:06',
     'ctime': '2025-10-03 16:51:06',
     'data_pool': 'cephfs.CEPH-FS-01.data',
     'earmark': '',
     'features': ['snapshot-clone',
      'snapshot-autoprotect',
      'snapshot-retention'],
     'flavor': 2,
     'gid': 0,
     'mode': 16895,
     'mon_addrs': ['10.129.254.2:6789',
      '10.135.254.2:6789',
      '10.129.253.2:6789'],
     'mtime': '2025-10-03 16:51:06',
     'path': '/volumes/_nogroup/kthare10_0011904101/454d15fd-e116-4af1-b743-f4bba6bfb8e5',
     'pool_namespace': '',
     'state': 'complete',
     'type': 'subvolume',
     'uid': 0}},
   'message': 'Subvolume CEPH-FS-01 deleted.'}],
 'size': 1,
 'status': 200,
 'type': 'no_content'}